<a href="https://colab.research.google.com/github/MrFinicheck/2.4json/blob/main/labs/%D0%9B%D0%A01_%D0%9C%D0%B0%D0%BD%D0%B8%D0%BF%D1%83%D0%BB%D1%8F%D1%86%D0%B8%D0%B8_%D1%81_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%87%D0%BD%D1%8B%D0%BC%D0%B8_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Постановка задачи


**Цель работы:** изучение основных способов обработки табличных данных и создания запросов к ним с помощью модуля `pandas`.

**Краткое описание:** в лабораторной работе представлено 8 заданий на сортировку, фильтрацию и группировку данных методами модуля `pandas`. Первые 4 задания являются обязательными. Из оставшихся 4 необходимо выполнить любые два на выбор обучающегося.

# Данные


В папке [Data/lab1](https://drive.google.com/drive/folders/1RxzrWKQC5zqStrKE_3TB9C3Hqzqf-7aJ?usp=share_link) расположено несколько таблиц с обезличенными транзакционными банковскими данными. Все задания лабораторной работы необходимо выполнять по этим данным.

### Таблица ```transactions.csv```
##### **Описание**
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### **Формат данных**

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### **Описание полей**

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком:
  - ```+``` — начисление средств клиенту (приходная транзакция);
  - ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала.


 ### Таблица ```gender_train.csv```

##### **Описание**
Таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### **Формат данных**
```
customer_id,gender
111111,0
111112,1
...
```

##### **Описание полей**
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента.

### Таблица ```tr_mcc_codes.csv```

##### **Описание**
Таблица содержит описание mcc-кодов транзакций.

##### **Формат данных**
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### **Описание полей**
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

### Таблица ```tr_types.csv```

##### **Описание**
Таблица содержит описание типов транзакций.

##### **Формат данных**
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### **Описание полей**
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции.

# Практические задания

In [ ]:
import pandas as pd
import numpy as np

## Подготовка

Загрузите и прочитайте данные из таблиц в переменные **tr_mcc_codes, tr_types, transactions** и **gender_train** из одноименных таблиц.

Выведите размеры таблиц и случайные строки из них.


Задания 1–4 нужно выполнить без использования функции `merge` и аналогичных по назначению.


>Допускается для таблицы **transactions** использование не всех записей. При чтении файлов обратите внимание на разделители внутри каждого из файлов – они могут различаться!


## Задание 1

1. Для столбца `tr_type` датафрейма `transactions` выберите произвольные `1000` строк с помощью метода `sample()`.
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец `tr_description` в датафрейме `tr_types`), в которой содержится подстрока `'POS'` или `'ATM'`.


In [2]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()

tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep=';')
tr_types = pd.read_csv('tr_types.csv', sep=';')
transactions = pd.read_csv('transactions.csv', sep=';')
gender_train = pd.read_csv('gender_train.csv', sep=';')

print("Размеры таблиц:")
print(f"tr_mcc_codes: {tr_mcc_codes.shape}")
print(f"tr_types: {tr_types.shape}")
print(f"gender_train: {gender_train.shape}")

print("\n" + "="*50 + "\n")

print("Случайные строки из tr_mcc_codes:")
print(tr_mcc_codes.sample(3))
print("\nСлучайные строки из tr_types:")
print(tr_types.sample(3))
print("\nСлучайные строки из gender_train:")
print(gender_train.sample(3))

print("\n" + "="*50 + "\n")

tr_type_sample = transactions['tr_type'].sample(1000, random_state=42)

tr_types_dict = dict(zip(tr_types['tr_type'], tr_types['tr_description']))

descriptions = []
for tr_type in tr_type_sample:
    if tr_type in tr_types_dict:
        descriptions.append(tr_types_dict[tr_type])
    else:
        descriptions.append(None)

# Подсчитываем долю транзакций, содержащих подстроку 'POS' или 'ATM'
count_pos_atm = 0
total = 0

for desc in descriptions:
    if desc is not None:
        total += 1
        if 'POS' in desc or 'ATM' in desc:
            count_pos_atm += 1

# Вычисляем долю
if total > 0:
    proportion = count_pos_atm / total
    print(f"Доля транзакций, содержащих 'POS' или 'ATM': {proportion:.4f} ({count_pos_atm} из {total})")
else:
    print("Нет данных для анализа")

print("\n" + "="*50 + "\n")


Saving tr_types.csv to tr_types (1).csv
Saving tr_mcc_codes.csv to tr_mcc_codes (1).csv
Saving gender_train.csv to gender_train (1).csv
Размеры таблиц:
tr_mcc_codes: (184, 2)
tr_types: (155, 2)
gender_train: (8400, 1)


Случайные строки из tr_mcc_codes:
     mcc_code                           mcc_description
60       5462                                  Булочные
166      8011  Доктора, нигде ранее не классифицируемые
145      7399                             Бизнес-сервис

Случайные строки из tr_types:
    tr_type                                     tr_description
46     7025  Взнос наличных через POS (в других ТБ) по счет...
72     8003          Списание подоходного налога с нерезидента
44     7021                           Взнос наличных через POS

Случайные строки из gender_train:
     customer_id,gender
8040         53779441,1
6425         29663240,1
7010         99387107,0




NameError: name 'transactions' is not defined

## Задание 2


1. Для столбца `tr_type` датафрейма `transactions` посчитайте частоту встречаемости всех типов транзакций `tr_type` в `transactions`.
2. Выведите датафрейм, содержащий топ-10 транзакций, отсортированных по убыванию частоты встречаемости. Датафрейм должен содержать столбцы `tr_type`, `tr_description` и столбец с частотой встречаемости.

In [ ]:
# Считаем частоту встречаемости типов транзакций
tr_type_counts = transactions['tr_type'].value_counts().reset_index()
tr_type_counts.columns = ['tr_type', 'frequency']

# Получаем топ-10 типов транзакций
top_10_tr_types = tr_type_counts.head(10)

# Добавляем описание транзакций
top_10_with_desc = []
for idx, row in top_10_tr_types.iterrows():
    tr_type = row['tr_type']
    frequency = row['frequency']

    # Ищем описание в tr_types
    description_row = tr_types[tr_types['tr_type'] == tr_type]
    if not description_row.empty:
        description = description_row.iloc[0]['tr_description']
    else:
        description = 'Неизвестно'

    top_10_with_desc.append({
        'tr_type': tr_type,
        'tr_description': description,
        'frequency': frequency
    })

  # Создаем итоговый датафрейм
result_df_2 = pd.DataFrame(top_10_with_desc)
print("Топ-10 типов транзакций по частоте:")
print(result_df_2)

print("\n" + "="*50 + "\n")

## Задание 3
1. В датафрейме `transactions` найдите клиента с максимальной суммой приходов на карту.
2. В датафрейме `transactions` найдите клиента с максимальной суммой расходов по карте.
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов.

In [ ]:
# Проверим структуру данных
print("Столбцы transactions:", transactions.columns.tolist())

if 'direction' in transactions.columns:
    # Приходы
    income_transactions = transactions[transactions['direction'] == 'income']
    income_by_client = income_transactions.groupby('client_id')['amount'].sum()
    max_income_client = income_by_client.idxmax()
    max_income = income_by_client.max()

    # Расходы
    expense_transactions = transactions[transactions['direction'] == 'expense']
    expense_by_client = expense_transactions.groupby('client_id')['amount'].sum()
    max_expense_client = expense_by_client.idxmin()  # idmin, так как расходы отрицательные
    max_expense = expense_by_client.min()

  #Если нет явного столбца направления
else:
    # Разделяем на приходы (положительные значения) и расходы (отрицательные значения)
    income_transactions = transactions[transactions['amount'] > 0]
    expense_transactions = transactions[transactions['amount'] < 0]

    # Суммируем по клиентам
    income_by_client = income_transactions.groupby('client_id')['amount'].sum()
    expense_by_client = expense_transactions.groupby('client_id')['amount'].sum()

    # Находим клиентов с максимальными значениями
    max_income_client = income_by_client.idxmax()
    max_income = income_by_client.max()

    # Для расходов берем минимальное значение (самое отрицательное)
    max_expense_client = expense_by_client.idxmin()
    max_expense = expense_by_client.min()

print(f"Клиент с максимальной суммой приходов: {max_income_client}, сумма: {max_income:.2f}")
print(f"Клиент с максимальной суммой расходов: {max_expense_client}, сумма: {max_expense:.2f}")

# Вычисляем модуль разницы между суммой расходов и приходов для этих клиентов
# Для клиента с максимальными приходами
if max_income_client in expense_by_client:
    expense_for_income_client = abs(expense_by_client[max_income_client])
else:
    expense_for_income_client = 0

diff_income_client = abs(max_income - expense_for_income_client)

# Для клиента с максимальными расходами
if max_expense_client in income_by_client:
    income_for_expense_client = income_by_client[max_expense_client]
else:
    income_for_expense_client = 0

diff_expense_client = abs(abs(max_expense) - income_for_expense_client)

print(f"Модуль разницы для клиента {max_income_client}: {diff_income_client:.2f}")
print(f"Модуль разницы для клиента {max_expense_client}: {diff_expense_client:.2f}")

# Находим максимальный модуль разницы
max_diff = max(diff_income_client, diff_expense_client)
print(f"Максимальный модуль разницы: {max_diff:.2f}")

print("\n" + "="*50 + "\n")

## Задание 4
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций из топ-10 в Задании 2.
1. Найдите среднее арифметическое и медиану по столбцу `amount` для каждого типа транзакций, совершенных клиентами из Задания 3.

In [ ]:
# Создаем словарь для хранения результатов
results_top_10 = []

for tr_type in result_df_2['tr_type']:
    # Фильтруем транзакции по типу
    type_transactions = transactions[transactions['tr_type'] == tr_type]

    if not type_transactions.empty:
        mean_amount = type_transactions['amount'].mean()
        median_amount = type_transactions['amount'].median()

        # Находим описание
        description = result_df_2[result_df_2['tr_type'] == tr_type].iloc[0]['tr_description']

        results_top_10.append({
            'tr_type': tr_type,
            'tr_description': description,
            'mean_amount': mean_amount,
            'median_amount': median_amount
        })

results_df_top_10 = pd.DataFrame(results_top_10)
print("Среднее и медиана amount для топ-10 типов транзакций:")
print(results_df_top_10)

print("\n" + "="*50)

# Часть 2: Для клиентов из задания 3
print("\nДля клиентов из задания 3:")

# Создаем словарь для хранения результатов по клиентам
results_clients = []

# Анализируем для каждого из клиентов
clients_to_analyze = [max_income_client, max_expense_client]

for client_id in clients_to_analyze:
    # Фильтруем транзакции клиента
    client_transactions = transactions[transactions['client_id'] == client_id]

    # Группируем по типу транзакции
    for tr_type in client_transactions['tr_type'].unique():
        type_transactions = client_transactions[client_transactions['tr_type'] == tr_type]

        if not type_transactions.empty:
            mean_amount = type_transactions['amount'].mean()
            median_amount = type_transactions['amount'].median()

            # Находим описание
            description_row = tr_types[tr_types['tr_type'] == tr_type]
            description = description_row.iloc[0]['tr_description'] if not description_row.empty else 'Неизвестно'

            results_clients.append({
                'client_id': client_id,
                'tr_type': tr_type,
                'tr_description': description,
                'mean_amount': mean_amount,
                'median_amount': median_amount
            })

results_df_clients = pd.DataFrame(results_clients)
print("Среднее и медиана amount для клиентов из задания 3:")
print(results_df_clients)

print("\n" + "="*50 + "\n")

## Подготовка для заданий 5–8

>**Из заданий 5-8 нужно выполнить два любых на выбор.**

Соедините датафрейм `transactions` со всеми остальными таблицами (`tr_mcc_codes`, `tr_types`, `gender_train`). Причем объединение с таблицей `gender_train` необходимо выполнить с помощью `left join`, а с оставшимися датафреймами – через `inner`.
После получения общей таблицы `gender_train`, `tr_types` и `tr_mcc_codes` можно удалить. В результате соединения датафреймов должно получиться `999584` строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата – отрицательное значение в столбце `amount`).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин.

>Обратите внимание, что для вычисления модуля разности не требуется точных знаний о том,
каким значением в таблице обозначены мужчины, а каким – женщины.

In [ ]:
# Проверяем наличие столбца gender
if 'gender' in transactions.columns:
    print("Столбец 'gender' найден")
    print(f"Уникальные значения в столбце gender: {transactions['gender'].unique()}")
    print(f"Количество NaN в столбце gender: {transactions['gender'].isna().sum()}")
else:
    # Если столбец назван иначе, найдем его
    gender_cols = [col for col in transactions.columns if 'gender' in col.lower()]
    print(f"Возможные столбцы с гендером: {gender_cols}")
    if gender_cols:
        gender_col = gender_cols[0]
        print(f"Используем столбец: {gender_col}")
        transactions.rename(columns={gender_col: 'gender'}, inplace=True)
    else:
        # Предположим, что столбец называется как в исходном gender_train
        # Обычно это 'gender' или 'sex'
        raise ValueError("Столбец с гендером не найден")

# 1. Модуль разницы между средними тратами женщин и мужчин
# Трата - отрицательное значение в столбце amount
expenses = transactions[transactions['amount'] < 0].copy()

# Разделяем по полу
# Сначала определим, какие значения представляют мужчин и женщин
gender_values = expenses['gender'].dropna().unique()
print(f"\nУникальные значения гендера: {gender_values}")

# Для автоматического определения будем использовать порядок
if len(gender_values) >= 2:
    # Берем два уникальных значения
    gender_1, gender_2 = gender_values[:2]

    # Средние траты для каждого пола
    mean_expenses_1 = expenses[expenses['gender'] == gender_1]['amount'].mean()
    mean_expenses_2 = expenses[expenses['gender'] == gender_2]['amount'].mean()

    # Модуль разницы
    diff_expenses = abs(mean_expenses_1 - mean_expenses_2)

    print(f"\nСредние траты для '{gender_1}': {mean_expenses_1:.2f}")
    print(f"Средние траты для '{gender_2}': {mean_expenses_2:.2f}")
    print(f"Модуль разницы средних трат: {diff_expenses:.2f}")

    # Сохраняем значения для использования в следующей части
    mean_expenses_dict = {gender_1: mean_expenses_1, gender_2: mean_expenses_2}
else:
    print("Недостаточно уникальных значений гендера для сравнения")

# 2. Модуль разницы между средними поступлениями у мужчин и женщин
# Поступления - положительные значения в столбце amount
incomes = transactions[transactions['amount'] > 0].copy()

if len(gender_values) >= 2:
    # Средние поступления для каждого пола
    mean_incomes_1 = incomes[incomes['gender'] == gender_1]['amount'].mean()
    mean_incomes_2 = incomes[incomes['gender'] == gender_2]['amount'].mean()

    # Модуль разницы
    diff_incomes = abs(mean_incomes_1 - mean_incomes_2)

    print(f"\nСредние поступления для '{gender_1}': {mean_incomes_1:.2f}")
    print(f"Средние поступления для '{gender_2}': {mean_incomes_2:.2f}")
    print(f"Модуль разницы средних поступлений: {diff_incomes:.2f}")

    # Сохраняем значения
    mean_incomes_dict = {gender_1: mean_incomes_1, gender_2: mean_incomes_2}

    # Итоговый результат
    print(f"\nИТОГ ЗАДАНИЯ 5:")
    print(f"Модуль разницы средних трат: {diff_expenses:.2f}")
    print(f"Модуль разницы средних поступлений: {diff_incomes:.2f}")

    # Дополнительная информация
    print(f"\nДополнительная статистика:")
    print(f"Всего транзакций: {len(transactions)}")
    print(f"Трат (отрицательных amount): {len(expenses)}")
    print(f"Поступлений (положительных amount): {len(incomes)}")
    print(f"Распределение по полу в тратах:")
    print(expenses['gender'].value_counts(dropna=False))
    print(f"\nРаспределение по полу в поступлениях:")
    print(incomes['gender'].value_counts(dropna=False))

print("\n" + "="*50 + "\n")


## Задание 6

1. Для каждого типа транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу `amount`) отдельно для мужчин и женщин (назовите ее `max_income`). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям `max_income`.
2. Среди типов транзакций, найденных в пункте 1, выделите те, которые встречаются одновременно и у мужчин, и у женщин.

In [ ]:
# Проверяем, что у нас есть данные о поле
if 'gender' not in transactions.columns or transactions['gender'].isna().all():
    print("Нет данных о поле для выполнения задания")
else:
    # 1. Для каждого типа транзакций рассчитаем максимальную сумму прихода на карту
    # отдельно для мужчин и женщин (из строго положительных сумм)

    # Фильтруем только положительные суммы (приходы)
    positive_transactions = transactions[transactions['amount'] > 0].copy()

    # Группируем по типу транзакции и полу, находим максимальную сумму прихода
    max_income_by_type_gender = positive_transactions.groupby(['tr_type', 'gender'])['amount'].max().reset_index()
    max_income_by_type_gender.rename(columns={'amount': 'max_income'}, inplace=True)

    print(f"\nУникальные значения полов в данных: {max_income_by_type_gender['gender'].dropna().unique()}")

    # Разделяем данные по полу
    genders = max_income_by_type_gender['gender'].dropna().unique()

    # Для каждого пола находим 10 типов транзакций с наименьшими max_income
    top_types_by_gender = {}

    for g in genders:
        # Фильтруем по полу
        gender_data = max_income_by_type_gender[max_income_by_type_gender['gender'] == g].copy()

        # Сортируем по max_income (по возрастанию) и берем 10 наименьших
        gender_data_sorted = gender_data.sort_values('max_income').head(10)

        # Добавляем описание транзакций
        # Создаем словарь описаний из исходных данных
        tr_type_desc_dict = {}
        for _, row in transactions[['tr_type', 'tr_description']].drop_duplicates().iterrows():
            tr_type_desc_dict[row['tr_type']] = row['tr_description']

        # Добавляем описание
        gender_data_sorted['tr_description'] = gender_data_sorted['tr_type'].map(tr_type_desc_dict)

        top_types_by_gender[g] = gender_data_sorted
        print(f"\nТоп-10 типов транзакций с наименьшими max_income для '{g}':")
        print(gender_data_sorted[['tr_type', 'tr_description', 'max_income']].to_string(index=False))

    # 2. Находим типы транзакций, которые встречаются одновременно у мужчин и женщин
    if len(genders) >= 2:
        # Получаем множества типов транзакций для каждого пола
        types_gender_1 = set(top_types_by_gender[genders[0]]['tr_type'])
        types_gender_2 = set(top_types_by_gender[genders[1]]['tr_type'])

        # Находим пересечение
        common_types = types_gender_1.intersection(types_gender_2)

        print(f"\nТипы транзакций, встречающиеся одновременно у '{genders[0]}' и '{genders[1]}':")
        if common_types:
            common_types_list = list(common_types)

            # Создаем датафрейм с информацией об общих типах
            common_types_info = []
            for tr_type in common_types_list:
                # Информация для первого пола
                info_g1 = top_types_by_gender[genders[0]][top_types_by_gender[genders[0]]['tr_type'] == tr_type]
                # Информация для второго пола
                info_g2 = top_types_by_gender[genders[1]][top_types_by_gender[genders[1]]['tr_type'] == tr_type]

                if not info_g1.empty and not info_g2.empty:
                    description = tr_type_desc_dict.get(tr_type, 'Неизвестно')
                    max_income_g1 = info_g1.iloc[0]['max_income']
                    max_income_g2 = info_g2.iloc[0]['max_income']

                    common_types_info.append({
                        'tr_type': tr_type,
                        'tr_description': description,
                        f'max_income_{genders[0]}': max_income_g1,
                        f'max_income_{genders[1]}': max_income_g2,
                        'difference': abs(max_income_g1 - max_income_g2)
                    })

            if common_types_info:
                common_df = pd.DataFrame(common_types_info)
                print(common_df.to_string(index=False))

                # Статистика
                print(f"\nСтатистика по общим типам транзакций:")
                print(f"Всего общих типов: {len(common_df)}")
                print(f"Средняя разница в max_income: {common_df['difference'].mean():.2f}")
                print(f"Минимальная разница: {common_df['difference'].min():.2f}")
                print(f"Максимальная разница: {common_df['difference'].max():.2f}")
            else:
                print("Нет общих типов транзакций в топ-10")
        else:
            print("Нет общих типов транзакций в топ-10")
    else:
        print(f"Недостаточно данных о разных полах (только {len(genders)} уникальных значения)")

    # Дополнительная визуализация (опционально)
    print("\n" + "="*50)
    print("ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ:")

    # Для каждого пола покажем распределение max_income
    for g in genders:
        if g in top_types_by_gender:
            data = top_types_by_gender[g]
            print(f"\nДля '{g}':")
            print(f"  Минимальный max_income: {data['max_income'].min():.2f}")
            print(f"  Максимальный max_income в топ-10: {data['max_income'].max():.2f}")
            print(f"  Средний max_income в топ-10: {data['max_income'].mean():.2f}")
            print(f"  Медиана max_income в топ-10: {data['max_income'].median():.2f}")

print("\n" + "="*50 + "\n")

## Задание 7

1. Найдите суммы затрат по каждой категории (`mcc`) для мужчин и для женщин.
2. Найдите топ-10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc-кодов).

In [ ]:
# PUT YOUR CODE HERE

## Задание 8

1. Из поля `tr_datetime` выделите час `tr_hour`, в который произошла транзакция, как первые 2 цифры до `":"`.
2. Посчитайте количество транзакций со значением в столбце `amount` строго меньше `0` в ночное время для мужчин и женщин. Ночное время – это промежуток с 00:00 по 06:00 часов.

In [ ]:
# PUT YOUR CODE HERE

# Порядок защиты работы

**Для защиты работы необходимо:**
1. Предоставить результаты выполнения 6 заданий в виде ссылки на Google Colab.
2. Выполнить дополнительное задание в процессе сдачи работы. Дополнительное задание представляет собой запрос к данным лабораторной работы, по сложности аналогичный тем, что требовалось выполнить в рамках работы.

**Примеры дополнительных заданий:**
1. Найдите категорию транзакций, в которой совершается больше всего трат в ночное время (промежуток с 00:00 по 06:00). Выведите название категории и среднюю сумму трат в ней.
2. Найдите 3 категории транзакций, в которых средние траты мужчин и женщин различаются меньше всего.
3. Найдите терминал, через который проходит наибольшее число транзакций. Выведите идентификаторы 5 пользователей с наибольшей суммой отрицательных транзакций в этом терминале.

# Примечание

Рекомендуется выполнение заданий двумя способами – с помощью `pandas` и с помощью `SQL` с использованием модуля `sqlite3`.

# Дополнительные материалы

1. Pandas. User Guide [Электронный ресурс]. URL: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html (дата обращения: 14.03.2024).
2. МакКинни, У. Python и анализ данных [Текст] / У. МакКинни ; пер. с англ. Слинкиной А. А. — Москва : ДМК-Пресс, 2023. — 536 с.
3. Хейдт, М. Изучаем pandas [Текст] / М. Хейдт. — Москва : ДМК Пресс, 2019. — 700 с.